### SPATIAL FEATURES AND SEPARATE REGRESSIONS

### Workflow:
1. Load final tract by day dataset (summer 2025).
2. Create additional interaction terms.
3. Split into high-heat vs. normal-heat categories.
4. OLS on log QoL rate and VIF checks for multicollinearity.
5. Moran's I on OLS residuals for spatial dependence.
6. Negative Binomial GLM count models for each categories.
7. Spatial regression.

For my reference because this is a long notebook.

In [ ]:
# Libraries.
import pandas as pd
import numpy as np
import geopandas as gpd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy
from esda.moran import Moran
import libpysal
import spreg
from pathlib import Path

In [ ]:
# Load final tract by day dataset (summer 2025).
df = pd.read_csv(
    "data/model/nyc_tract_daily_qol_heat_2025_final.csv",
    parse_dates = ["date"],
    dtype = {"GEOID": str}
)

print("Dataset:", df.shape)
df.head()

In [ ]:
# Build spatial weights (Queen contiguity) at tract level.

gdf_tracts = gpd.read_file("data/nyc_tracts_2020/nyc_tracts_2020.shp")

nyc_prefixes = ("36005", "36047", "36061", "36081", "36085")

gdf_tracts = gdf_tracts[gdf_tracts["GEOID"].str.startswith(nyc_prefixes)].copy()

# Keep only tracts that appear in df.
gdf_tracts = gdf_tracts[gdf_tracts["GEOID"].isin(df["GEOID"].unique())].copy()
gdf_tracts = gdf_tracts.reset_index(drop = True)

# Queen contiguity weights.
w = libpysal.weights.Queen.from_dataframe(gdf_tracts, idVariable = "GEOID")
w.transform = "r"

print("Number of tracts in W:", w.n)

In [ ]:
# Create interaction terms and QoL rate.
# QoL rate per total calls within each tract-day.
df["qol_rate"] = df["qol_calls"] / df["total_calls"].replace(0, np.nan)
df["log_qol_rate"] = np.log(df["qol_rate"] + 1e-6) # Add small number to account for 0.

# Centered environment variables if not in df.
if "ndvi_c" not in df.columns and "ndvi_mean" in df.columns:
    df["ndvi_c"] = df["ndvi_mean"] - df["ndvi_mean"].mean()

if "impervious_c" not in df.columns and "impervious_mean" in df.columns:
    df["impervious_c"] = df["impervious_mean"] - df["impervious_mean"].mean()

if "tree_canopy_fraction" in df.columns:
    df["tree_canopy_c"] = df["tree_canopy_fraction"] - df["tree_canopy_fraction"].mean()
else:
    df["tree_canopy_c"] = np.nan

# Interactions with TEMP_MAX since EXTREME_HEAT is constant within categories.
df["temp_max_x_poverty"] = df["temp_max_city_f"] * df["poverty_rate_c"]
df["temp_max_x_impervious"] = df["temp_max_city_f"] * df["impervious_c"]
df["temp_max_x_tree"] = df["temp_max_city_f"] * df["tree_canopy_c"]

In [ ]:
# Split into high-heat vs normal-heat days.
df_high = df[df["EXTREME_HEAT"] == 1].copy()
df_norm = df[df["EXTREME_HEAT"] == 0].copy()

print("High-heat rows:", len(df_high))
print("Normal-heat rows:", len(df_norm))

In [ ]:
# Variable blocks by category.

# Heat / 311 / temporal.
heat_vars = [
    "temp_max_city_f" # Daily max temp at JFK.
]

# 5.2 ACS / socioeconomic.
acs_vars = [
    "poverty_rate_c", # Poverty rate centered.
    "medhhinc_c", # Median income centered.
    "pct_bachelors_plus", # Percent bachelor's or more.
    "pct_renters", # Percent renters.
    "pct_limited_english", # Percent limited English speakers.
    "no_vehicle_rate" # Households without a vehicle rate.
]

# Environmental raster.
env_vars = [
    "ndvi_c", # NDVI centered.
    "ndwi_mean", # NDWI centered.
    "impervious_c", # Impervious centered.
    "tree_canopy_fraction", # Tree canopy fraction.
    "building_coverage", # Building coverage.
    "ndbi_mean" # NDBI mean.
]

# Spatially engineered.
spatial_vars = [
    "poverty_rate_lag" # Poverty rate lag.
]

# Interactions.
interaction_vars = [
    "temp_max_x_poverty", # Temperature * poverty.
    "temp_max_x_impervious", # Temperature * impervious.
    "temp_max_x_tree" # Temperature * tree canopy.
]

# Fixed effects for GLM via patsy?
# Placeholder for potential fixed effects.

# Polynomial?
# Placeholder for potential polynomials.

In [ ]:
# Loop to help with statistical modeling.
blocks = [
    ("heat_only", heat_vars),
    ("heat_acs", heat_vars + acs_vars),
    ("heat_acs_env", heat_vars + acs_vars + env_vars),
    ("heat_acs_env_spatial", heat_vars + acs_vars + env_vars + spatial_vars),
    ("full", heat_vars + acs_vars + env_vars + spatial_vars + interaction_vars)
]

In [ ]:
# High heat OLS.
for label, vars_used in blocks:
    vars_used = [v for v in vars_used if v in df_high.columns]

    formula = "log_qol_rate ~ " + " + ".join(vars_used) + " + C(dow)"
    print("\n--- Model:", label)
    print("Formula:", formula)

    ols_res = smf.ols(formula = formula, data = df_high).fit()
    print(ols_res.summary())

In [ ]:
# Normal heat OLS.
for label, vars_used in blocks:
    vars_used = [v for v in vars_used if v in df_norm.columns]

    formula = "log_qol_rate ~ " + " + ".join(vars_used) + " + C(dow)"
    print("\n--- Model:", label)
    print("Formula:", formula)

    ols_res = smf.ols(formula = formula, data = df_norm).fit()
    print(ols_res.summary())

In [ ]:
# High heat VIF.
X = df_high[heat_vars + acs_vars + env_vars + spatial_vars + interaction_vars].dropna()
X = sm.add_constant(X)

vif_df = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})

print(vif_df)

In [ ]:
# Normal heat VIF.
X = df_norm[heat_vars + acs_vars + env_vars + spatial_vars + interaction_vars].dropna()
X = sm.add_constant(X)

vif_df = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})

print(vif_df)

In [ ]:
# Moran's I on high heat OLS residuals.
ols_full_high = smf.ols(
    "log_qol_rate ~ " + " + ".join(blocks[-1][1]) + " + C(dow)",
    data = df_high
).fit()

df_high["ols_resid"] = ols_full_high.resid

# aggregate by tract
resid_mean_high = df_high.groupby("GEOID_TRACT")["ols_resid"].mean()

gdf_high = gdf.merge(resid_mean_high, on = "GEOID_TRACT")
mi_high = Moran(gdf_high["ols_resid"].values, w)

print("\nMoran's I: High Heat")
print("I =", mi_high.I)
print("p-value =", mi_high.p_sim)

In [ ]:
# Moran's I on normal heat OLS residuals.
ols_full_norm = smf.ols(
    "log_qol_rate ~ " + " + ".join(blocks[-1][1]) + " + C(dow)",
    data = df_norm
).fit()

df_norm["ols_resid"] = ols_full_norm.resid

resid_mean_norm = df_norm.groupby("GEOID_TRACT")["ols_resid"].mean()
gdf_norm = gdf.merge(resid_mean_norm, on = "GEOID_TRACT")
mi_norm = Moran(gdf_norm["ols_resid"].values, w)

print("\nMoran's I: Normal Heat")
print("I =", mi_norm.I)
print("p-value =", mi_norm.p_sim)

In [ ]:
# Prepare tract mean data.
agg_vars = ["log_qol_rate"] + heat_vars + acs_vars + env_vars
agg_vars = [v for v in agg_vars if v in df.columns]

tract_high = df_high.groupby("GEOID_TRACT")[agg_vars].mean().reset_index()
tract_norm = df_norm.groupby("GEOID_TRACT")[agg_vars].mean().reset_index()

In [ ]:
# Align weights.
aligned_high = gdf[["GEOID_TRACT"]].merge(tract_high, on = "GEOID_TRACT", how = "left").dropna()
aligned_norm = gdf[["GEOID_TRACT"]].merge(tract_norm, on = "GEOID_TRACT", how = "left").dropna()

y_high = aligned_high["log_qol_rate"].values.reshape(-1,1)
X_high = aligned_high[heat_vars + acs_vars + env_vars].values

y_norm = aligned_norm["log_qol_rate"].values.reshape(-1,1)
X_norm = aligned_norm[heat_vars + acs_vars + env_vars].values

In [ ]:
print("\nSpatial Error: High Heat")
sem_high = spreg.GM_Error_Het(y_high, X_high, w = w)
print(sem_high.summary)

print("\nSpatial Error: Normal Heat")
sem_norm = spreg.GM_Error_Het(y_norm, X_norm, w = w)
print(sem_norm.summary)

In [ ]:
# High heat NB regression.
formula_nb = (
    "qol_calls ~ "
    + " + ".join(blocks[-1][1])
    + " + C(dow) + C(GEOID_TRACT)"
)

y, X = patsy.dmatrices(formula_nb, df_high, return_type = "dataframe")

nb_high = sm.GLM(
    y,
    X,
    family = sm.families.NegativeBinomial(),
    offset = df_high["log_total_calls"]
).fit()

print("\nNB Model: High Heat")
print(nb_high.summary())

In [ ]:
# Normal heat NB regression.
y, X = patsy.dmatrices(formula_nb, df_norm, return_type = "dataframe")

nb_norm = sm.GLM(
    y,
    X,
    family = sm.families.NegativeBinomial(),
    offset = df_norm["log_total_calls"]
).fit()

print("\nNB Model: Normal Heat")
print(nb_norm.summary())